In [ ]:
!pip install textblob

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

# Vader 분석기 초기화
analyzer = SentimentIntensityAnalyzer()

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')


def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www|\S+https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)



stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data


train.text = train['text'].apply(data_processing)
train['text'] = train['text'].apply(lambda x: stemming(x))
test.text = test['text'].apply(data_processing)
test['text'] = test['text'].apply(lambda x: stemming(x))


Y_train = train['sentiment']

X_test = test['text']

# 각 행의 sentiment 값에 따라 처리
filtered_sentences = []

for index, row in train.iterrows():
    text = row['text']
    sentiment = row['sentiment']
    
    if sentiment != 2:  # sentiment 값이 2가 아닌 경우
        filtered_sentences.append(text)
    else:  # sentiment 값이 2인 경우 (부정인 경우)
        words = text.split()
        negative_words = [word for word in words if analyzer.polarity_scores(word)['compound'] < 0]
        if negative_words:
            filtered_sentence = ' '.join(negative_words)
            filtered_sentences.append(filtered_sentence)
        else:
            filtered_sentences.append("")

# 결과 출력 (판다스 시리즈로 변환)
X_train = pd.Series(filtered_sentences)
print(X_train)


In [ ]:
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www|\S+https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)



stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data



train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')



train.text = train['text'].apply(data_processing)
train['text'] = train['text'].apply(lambda x: stemming(x))
test.text = test['text'].apply(data_processing)
test['text'] = test['text'].apply(lambda x: stemming(x))




X_train = train['text']
Y_train = train['sentiment']

X_test = test['text']



## X_train 토큰화

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

text_series = X_train
token_list = []
for text in text_series:
    tokens = tokenize_text(text)
    token_list.append(tokens)

In [ ]:
X_train

In [ ]:
train

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

# Vader 분석기 초기화
analyzer = SentimentIntensityAnalyzer()


# 각 행의 sentiment 값에 따라 처리
filtered_sentences = []

for index, row in train.iterrows():
    text = row['text']
    sentiment = row['sentiment']
    
    if sentiment != 2:  # sentiment 값이 2가 아닌 경우
        filtered_sentences.append(text)
    else:  # sentiment 값이 2인 경우 (부정인 경우)
        words = text.split()
        negative_words = [word for word in words if analyzer.polarity_scores(word)['compound'] < 0]
        if negative_words:
            filtered_sentence = ' '.join(negative_words)
            filtered_sentences.append(filtered_sentence)
        else:
            filtered_sentences.append("")

# 결과 출력 (판다스 시리즈로 변환)
result_series = pd.Series(filtered_sentences)
print(result_series)


In [ ]:
train = train.drop(['id'],axis=1)
train.groupby('sentiment').count().plot.bar()

In [ ]:
tokens = pd.Series(token_list)

frequencies = pd.Series(np.concatenate([w for w in token_list])).value_counts()
frequencies

In [ ]:

def get_freq_count(tokens):
    neg_token = tokens[train['sentiment'] == 2]
    pos_token = tokens[train['sentiment'] == 1]
    neul_token = tokens[train['sentiment'] == 0]
    neg_freq = pd.Series(np.concatenate([w for w in neg_token])).value_counts()
    pos_freq = pd.Series(np.concatenate([w for w in pos_token])).value_counts()
    neul_freq = pd.Series(np.concatenate([w for w in neul_token])).value_counts()
    return neg_freq, pos_freq, neul_freq

def remove_doubled_words(neg_freq ,pos_freq, neul_freq, tokens):
    top_50_neg = neg_freq[:20]
    top_50_pos = pos_freq[:20]
    top_50_neul = neul_freq[:20]
    remove_word = [p for p in top_50_pos.index if p in top_50_neg.index]
    tokens_removed = remove_stop_words(tokens, remove_word)
    final_tokens = cleaning_tokens(tokens_removed)
    return final_tokens


def draw_top_hist(data,name,color): 
    top_10 = data[:10]
    plt.figure(figsize=(10, 5))
    plt.suptitle("Bar Plot", fontsize=30)
    plt.rc('font', family='Arial')
    plt.title(name, fontsize=15)
    plt.bar(top_10.index, top_10.values, width=0.5, color=color, alpha=0.5)
    plt.xticks(top_10.index, fontsize=15)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
neg_freq ,pos_freq, neul_freq = get_freq_count(tokens)

In [ ]:
top_50_pos = pos_freq[:3]
top_50_neg = neg_freq[:3]
top_50_neul = neul_freq[:3]
common_words = [p for p in top_50_neg.index if p in set.intersection(set(top_50_pos.index), set(top_50_neul.index))]
common_words

In [ ]:
clean_token_list = []
common_neg_words = ['like', 'get', 'go']

for i in range(len(tokens)):
    token = tokens.iloc[i]  # 각 토큰 가져오기
    sentiment = train['sentiment'].iloc[i]  # 해당 토큰의 sentiment 값 가져오기
    
    # 해당 토큰의 sentiment가 2인 경우 common_neg_words를 사용하여 필터링
    if sentiment == 2:
        clean_token_neg = list(filter(lambda x: x not in common_neg_words, token))
        clean_token = list(filter(lambda x: x not in common_words, clean_token_neg))
    else:
        clean_token = list(filter(lambda x: x not in common_words, token))
    
    clean_token_list.append(clean_token)

clean_tokens = pd.Series(clean_token_list)

In [ ]:
clean_tokens

In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

tagged_tokens = clean_tokens.apply(lambda tokens: pos_tag(tokens))

In [ ]:
## CD(숫자를 나타내는 품사), NNP(고유명사. 단수형), NNPS(고유명사, 복수형) tag 제거 후 Series객체로 반환

filtered_series = [[word for word, tag in tagged_tokens if tag not in ['CD','NNP','NNPS']] for tagged_tokens in tagged_tokens]
filtered_series = pd.Series(filtered_series)


## 토큰화 되어있는 Series객체 토큰들 합치기

X_train = filtered_series.apply(lambda tokens:' '.join(tokens))

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train_vec, Y_train, test_size=0.5, random_state=42)

In [ ]:
# 랜덤 포레스트 분류기
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# 그래디언트 부스팅 분류기
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)

In [ ]:
# 랜덤 포레스트 분류기
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# 그래디언트 부스팅 분류기
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)

ensemble_classifier = VotingClassifier(estimators=[
    ('rf', rf_classifier),
    ('gb', gb_classifier)
], voting='hard')

In [ ]:
ensemble_classifier.fit(x_train, y_train)

In [ ]:
# 예측 확률 계산 함수
def get_emotion_probabilities(text, model, vectorizer):
    # 텍스트 데이터를 TF-IDF 벡터로 변환
    text_tfidf = vectorizer.transform([text])
    
    # 각 클래스(감정)에 대한 확률 예측
    probabilities = model.predict_proba(text_tfidf)
    
    # 클래스에 따른 확률을 딕셔너리로 반환
    emotion_probabilities = {
        "중립": probabilities[0][0],
        "긍정": probabilities[0][1],
        "부정": probabilities[0][2]
    }
    
    return emotion_probabilities

# 텍스트 감정 분류 예측 예제
new_text = "like"
predicted_emotion_probabilities = get_emotion_probabilities(new_text, ensemble_classifier, vectorizer)

# 예측된 각각의 확률 출력
for emotion, probability in predicted_emotion_probabilities.items():
    print(f"{emotion} 확률: {probability}")

In [ ]:
Y_val_pred = ensemble_classifier.predict(x_test)
print(classification_report(y_test, Y_val_pred))

In [ ]:
Y_test_pred = ensemble_classifier.predict(X_test_vec)

# 결과 저장
submit = pd.read_csv('./sample_submission.csv')
submit['sentiment'] = Y_test_pred
submit.to_csv('./ensemble_sentiment17.csv', index=False)

print('Done')

In [ ]:
stop_words

In [ ]:
import numpy as np

In [ ]:
a = [1, 2, 3, 4, 5]

In [ ]:
a